In [ ]:
#GENERATE EXPERT ANSWERS

answer_instructions = """You are a culinary expert being interviewed.

Analyst focus: {goals}

Use ONLY the provided sources to answer: {context}

Guidelines:
1. Base answers on evidence from provided sources
2. Cite sources using [1], [2], etc.
3. Be specific and practical
4. Include cooking tips
5. Keep response concise (max 500 words)

Provide accurate, helpful culinary information."""

def generate_answer(state: InterviewState):
    analyst = state["analyst"]
    messages = state["messages"]
    context = state["context"]

    # Limit context
    context_text = '\n'.join(context)
    if len(context_text) > 3000:
        context_text = context_text[:3000] + "...[truncated]"

    system_message = answer_instructions.format(goals=analyst.persona, context=context_text)
    answer = llm.invoke(
        [SystemMessage(content=system_message)] + messages,
        max_tokens=1000  # Limit response
    )
    answer.name = "culinary_expert"

    return {"messages": [answer]}

def save_interview(state: InterviewState):
    messages = state["messages"]
    interview = get_buffer_string(messages)
    return {"interview": interview}

def route_messages(state: InterviewState, name: str = "culinary_expert"):
    messages = state["messages"]
    max_num_turns = state.get('max_num_turns', 2)

    num_responses = len([m for m in messages if isinstance(m, AIMessage) and m.name == name])

    if num_responses >= max_num_turns:
        return 'save_interview'

    if len(messages) >= 2:
        last_question = messages[-2]
        if "Thank you so much for your help" in last_question.content:
            return 'save_interview'

    return "ask_question"

print(" Answer generation system ready!")

In [ ]:
#BUILD INTERVIEW GRAPH

interview_builder = StateGraph(InterviewState)
interview_builder.add_node("ask_question", generate_question)
interview_builder.add_node("search_web", search_web)
interview_builder.add_node("search_wikipedia", search_wikipedia)
interview_builder.add_node("answer_question", generate_answer)
interview_builder.add_node("save_interview", save_interview)

interview_builder.add_edge(START, "ask_question")
interview_builder.add_edge("ask_question", "search_web")
interview_builder.add_edge("ask_question", "search_wikipedia")
interview_builder.add_edge("search_web", "answer_question")
interview_builder.add_edge("search_wikipedia", "answer_question")
interview_builder.add_conditional_edges("answer_question", route_messages, ["ask_question", "save_interview"])
interview_builder.add_edge("save_interview", END)

interview_graph = interview_builder.compile()

print(" Interview graph compiled!")